In [5]:
#Dependencies
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession
from lxml import etree
from requests_html import HTML



In [2]:
#utilts
import os
import json
def printSoupToHtml(soup):
    #Creating a file if not exist
    if not os.path.exists('soup.html'):
        with open('soup.html', 'w', encoding='utf-8') as file:
            file.write('')
    with open('soup.html', 'w', encoding='utf-8') as file:
        file.write(str(soup))
    return True
def printSoupToJson(soup):
    #Creating a file if not exist
    if not os.path.exists('soup.json'):
        with open('soup.json', 'w', encoding='utf-8') as file:
            file.write('')
    with open('soup.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(soup))
    return True

In [53]:

def getStringBetweenTwoWords(string, first, last):
    try:
        start = string.index(first) + len(first)
        end = string.index(last, start)
        return string[start:end]
    except ValueError:
        return ""
def convertStringToJson(string):
    x = json.dumps(string)
    printSoupToJson(x)

async def scrapeJarir(url):  
    session = AsyncHTMLSession()
    response = await session.get(url)
    script_target_object=getStringBetweenTwoWords(response.content.decode(), 'window.__INITIAL_STATE__= ','</script>')
    strat_index=response.content.decode().index('window.__INITIAL_STATE__')
    end_index=response.content.decode().index('</script>',strat_index)
    script_target_object=response.content.decode()[strat_index:end_index].replace('window.__INITIAL_STATE__=','')
    printSoupToHtml(script_target_object)
    price=getStringBetweenTwoWords(script_target_object, 'final_price_ex_tax:',',')
    Title=getStringBetweenTwoWords(script_target_object, 'GTM_name:',',')
    images_string = getStringBetweenTwoWords(script_target_object, 'media_gallery','tsk').split("},{")
    #[{type:"image",image:"b/b/c/c/bbcc8c1c12a995f33ba9a9b8ab7c365703114777_631416.jpg",lab:"",pos:0},{type:"image",image:"f/4/f/e/f4fedbefb6cb954f43d7f6125bdbe259d7cf4d07_631416_1.jpg",lab:"",pos:1},{type:"image",image:"1/f/6/0/1f60183c0fc8f48082251e64b74fcc4dc7c74378_631416_2.jpg",lab:"",pos:2},{type:"image",image:"7/2/c/b/72cb5a45081b0696c661de124c2e0bb4758f2128_631416_3.jpg",lab:"",pos:3},{type:"image",image:"5/3/8/6/5386b16272b56e1c79cb085f7afc2167dacc6308_631416_4.jpg",lab:"",pos:4}],')
    images=[]
    for i in range(len(images_string)):
        images.append(getStringBetweenTwoWords(images_string[i], 'image:"','",lab'))
    return price , Title,images


In [54]:
#Test
url = 'https://www.jarir.com/sa-en/default-category/msi-clae-a1m-gaming-consoles-and-handheld-631416.html'
print( await scrapeJarir(url))

('3303.4783', '"MSI Claw A1M Handheld Gaming Console', ['b/b/c/c/bbcc8c1c12a995f33ba9a9b8ab7c365703114777_631416.jpg', 'f/4/f/e/f4fedbefb6cb954f43d7f6125bdbe259d7cf4d07_631416_1.jpg', '1/f/6/0/1f60183c0fc8f48082251e64b74fcc4dc7c74378_631416_2.jpg', '7/2/c/b/72cb5a45081b0696c661de124c2e0bb4758f2128_631416_3.jpg', '5/3/8/6/5386b16272b56e1c79cb085f7afc2167dacc6308_631416_4.jpg'])
